# Amplitude (from MARTY) to Tree
In this notebook I show off how I convert amplitudes to trees.

The amplitudes are already in a modified format exported through a cpp program.

In [1]:
import sys
import os
from icecream import ic 
import numpy as np
import sympy as sp
from nltk.tree import Tree

In [8]:
from source.ampl_to_tree import ampl_to_tree, raw_ampl_to_tree, expand_tree, contract_tree, subscripts_to_subtree

# Full Amplitude

In [16]:
# this is how it is exported from the cpp program
expr_raw = "Prod;(;-1/2;i;Pow;(;e;2;);Pow;(;Sum;(;Pow;(;m_e;2;);Prod;(;-1;s_13;);Prod;(;1/2;reg_prop;););-1;);gamma_{+%\sigma_126,%eps_36,%del_171};gamma_{%\sigma_126,%eta_132,%del_172};e_{i_3,%del_171}(p_1)_u;e_{k_3,%del_172}(p_2)_u;e_{l_3,%eps_36}(p_3)_u^(*);e_{i_5,%eta_132}(p_4)_u^(*);)"
tree = raw_ampl_to_tree(expr_raw)
tree.pretty_print(unicodelines=True, abbreviate=False)

                                                                                                    Prod                                                                                                    
 ┌────┬───────┬───────────┬────────────────────────────────────────────────────────┬─────────────────┴─────────┬──────────────────────┬───────────────────┬───────────────────┬──────────────────┐           
 │    │       │          Pow                                                       │                           │                      │                   │                   │                  │          
 │    │       │       ┌───┴───────────┐                                            │                           │                      │                   │                   │                  │           
 │    │       │       │              Sum                                           │                           │                      │                   │                   │   

# Explanation of parts

## Products and sums

In [4]:
expr_raw = "Prod;(;1/9;Sum;(;a;b;c;);i;Pow;(;e;3;);)"
tree = raw_ampl_to_tree(expr_raw)
tree.pretty_print(unicodelines=True)

            Prod                
 ┌───┬───────┼────────────┐      
 │   │      Sum          Pow    
 │   │   ┌───┼────┐   ┌───┴───┐  
1/9  i   a   b    c   e       3 



## Indices

In MARTY indices have a % in front and I am keeping this, since I think it's a good idea.

$\gamma^a_{bc}$  turns into a subtree where `gamma` is the node and a, b, c are the leaves.  
Similar for the basis functions:  
- $e_u^{i, \delta}(p_1)$ turns into a subtree where `e_u` is the node and `i`, `delta` and `p_1` are the leaves.
- if there is a complex conjugate, $e_u^{i, \delta}(p_1)^*$  then the star is added to the node name.

For momenta only the Lorentz index is made to a leaf, not the number, i.e.
$p_1^\mu$ becomes `p_1` with `%mu` as leaf.

In [18]:
expr_1 = "gamma_{a,b,c}"
expr_2 = "gamma_{%\\sigma_126,%eta_132,%del_172}"
expr_3 = "e_{i_3,%del_171}(p_1)_u"
expr_4 = "p_1_mu"
subscripts_to_subtree(expr_1).pretty_print(unicodelines=True)
subscripts_to_subtree(expr_2).pretty_print(unicodelines=True)
subscripts_to_subtree(expr_3).pretty_print(unicodelines=True)
subscripts_to_subtree(expr_4).pretty_print(unicodelines=True)

    gamma    
 ┌────┼────┐  
 %a   %b   %c

            gamma           
    ┌─────────┼────────┐     
%sigma_126 %eta_132 %del_172

       e_u         
 ┌──────┼───────┐   
%i_3 %del_171 (p_1)

p_1
 │  
%mu



In [19]:
# in a full expression this is done automatically

expr_1 = "Sum;(;1;gamma_{a,b,c};)"
expr_2 = "Sum;(;1;gamma_{%\\sigma_126,%eta_132,%del_172};)"
expr_3 = "Sum;(;1;e_{i_3,%del_171}(p_1)_u;)"
ret_1 = raw_ampl_to_tree(expr_1)
ret_2 = raw_ampl_to_tree(expr_2)
ret_3 = raw_ampl_to_tree(expr_3)
ret_1.pretty_print(unicodelines=True, abbreviate=True)
ret_2.pretty_print(unicodelines=True, abbreviate=True)
ret_3.pretty_print(unicodelines=True, abbreviate=True)

        Sum          
 ┌───────┴────┐       
 │          gamma    
 │   ┌────────┼────┐  
 1   %a       %b   %c

           Sum              
 ┌──────────┴────┐           
 │             gamma        
 │    ┌──────────┼──────┐    
 1  %sigm…     %eta_… %del_…

         Sum             
 ┌────────┴────┐          
 │            e_u        
 │   ┌─────────┼──────┐   
 1  %i_3     %del_… (p_1)



# Expanding Trees
The tree can be "expanded" if it is needed that a node has a fixed number of leaves, e.g.
if `Prod` should only always have exactly two arguments, then this expands it.
The inverse I call "contract". (Not sure if the word "contract" is used with trees in a different manner, though.)

In [20]:
expr_raw = "Prod;(;1/9;Sum;(;a;b;c;);i;Pow;(;e;3;);)"
tree = raw_ampl_to_tree(expr_raw)
tree.pretty_print(unicodelines=True)
tree = expand_tree(tree)
tree.pretty_print(unicodelines=True)
tree = contract_tree(tree, add_opening_bracked=False)
tree.pretty_print(unicodelines=True)

            Prod                
 ┌───┬───────┼────────────┐      
 │   │      Sum          Pow    
 │   │   ┌───┼────┐   ┌───┴───┐  
1/9  i   a   b    c   e       3 

    Prod                              
 ┌───┴────────────┐                    
 │               Prod                 
 │        ┌───────┴────────┐           
 │       Sum              Prod        
 │   ┌────┴───┐        ┌───┴────┐      
 │   │       Sum       │       Pow    
 │   │    ┌───┴───┐    │   ┌────┴───┐  
1/9  a    b       c    i   e        3 

            Prod                
 ┌───┬───────┼────────────┐      
 │   │      Sum(         Pow    
 │   │   ┌───┼────┐   ┌───┴───┐  
1/9  i   a   b    c   e       3 

